In [ ]:

from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:

import os
import json
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if not openai_api_key:
    print('Missing OpenaAI API key.')
if not deepseek_api_key:
    print('Missing Deepseek API key')
if openai_api_key and deepseek_api_key:
    print(f'OpenAI: {openai_api_key[-10:]}\n')
    print(f'Deepseek: {deepseek_api_key[-10:]}\n')


In [ ]:
openai = OpenAI()

In [ ]:

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [ ]:
# systemp prompt form entrepreneur 
system_prompt = """
You're an entrepreneur looking to develop and invest in emerging AI-driven small SaaS applications that are easy to implements, 
and solve huge numbers of users' pain point for small business. 
You focus on solutions that use agentic AI to solve real-world small business problems with minimal human oversight,
offering scalability and innovation. Your goal is to find user pain points for small businesses, improving services
for business and users, while being user-friendly and ready to scale.
You select the the idea that is easy to to implement in a short amount of time. 
"""

user_prompt = "Pick a business area that might be worth exploring for Agentic AI, Respond with just the business area name. "

messages = [
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": user_prompt},
]

# Call openai
response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

# Then read the business idea:

business_idea = response.choices[0].message.content
print(business_idea)

# And repeat! In the next message, include the business idea within the message

In [ ]:
app = []

In [ ]:
app.append(
    {"business_idea": business_idea}
    )

In [ ]:
app

In [ ]:
user_second_prompt = f"Find user pain points in {business_idea} areas. Give reasons and why this pain point is the best to solve. List the top 1."

second_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_second_prompt}
]

In [ ]:
response = openai.chat.completions.create(
    messages=second_messages,
    model="gpt-4.1-mini"
)

pain_point = response.choices[0].message.content
print(pain_point)

In [ ]:
app.append({"pain_point": pain_point})

In [ ]:
app

In [ ]:
third_user_prompt =f"""Please create a react website file directory structure. You're given the following pain points 
we need to solve, for small business\n. {pain_point}. Respond in json format. The structure should also include strip 
payment."""

third_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": third_user_prompt}
]

response  = openai.chat.completions.create(
    messages=third_messages,
    model="gpt-4.1-mini"    
)
structure = response.choices[0].message.content
display(Markdown(structure))


In [ ]:
app.append({"app_structure": structure})

In [ ]:
app

In [ ]:
# We have all the info we need to create the application,let's call deepseek to implement a section of the app
user_app_prompt = f"""You're given a business idea, along with user pain points, and app structure. 
Please write the source code for the components directory files in\n {app}"""

In [ ]:
system_prompt = "You're a react app developer"

In [ ]:
messages = [
    {"role": "system", "content" : system_prompt},
    {"role": "user", "content": user_app_prompt}
]

In [ ]:
response = openai.chat.completions.create(
    messages=messages,
    model="gpt-4.1"
)

source = response.choices[0].message.content
if source:
    display(Markdown(source))
else:
    print("Error: failed to generate code.")
    


In [49]:
# Confirm with deepseek if the source code is correct 

tester_system_prompt = """You're react source code tester. 
You make sure the code is error free, along with smooth navigation."""

tester_user_prompt = """You're given a business idea, user pain points, app structure, and the  source code in components section
\n
{app}
\n
{source}
Please test the source code and check the components files only. Does it solve user pain points? Are there any errors through navigation, inputs or missing files.
Find all the mistakes in the source code. If everything works, respond with 'Correct', if not, then 
simply state the mistake, error with a quick explanation. 
"""

response = openai.chat.completions.create(
    messages=[
        {"role": "user", "content": tester_system_prompt},
        {"role": "user", "content": tester_user_prompt }
    ],
    model="gpt-4.1",
    stream=False
)
results = response.choices[0].message.content
if results:
    display(Markdown(results))

Of course! Please provide:

- The business idea  
- User pain points  
- App structure  
- Source code (components section)  

Once you provide these, I'll review the components for errors and articulate any issues plainly, as instructed.

In [ ]:
# Ask openai if it can see the mistakes what deepseek says. 
response = openai.chat.completions.create(
    messages=[
        {"role":"assistant", "content": "You're a react source code tester"},
        {"role": "user", "content": f"Please check if there are errors in {source}. The following are some errors {results}"}
    ],
    model="gpt-4.1"
)

tester_response = response.choices[0].message.content
display(Markdown(tester_response))

Here is a thorough review of your provided **React** components code for errors & common issues.  
**Assumptions:**  
- You are not using TypeScript.
- All context/hooks (`BookingContext`, `useAI`, etc.) exist and work as expected.
- The placeholder logic is accepted as-is.

---

## **General Observations**

- **Imports**: All essential React imports are present.
- **Props**: Most prop passing is correct, though in places you are passing structures that likely need shape clarification or refinement.
- **Handlers**: Some handler signatures may not match input children (see BookingForm section).
- **Optional Chaining**: Used where appropriate.
- **Styling classes**: You use `className`, all is correct.
- **Key prop**: Present on lists.
- **Context**: Usage assumes a correctly configured provider.
- **Controlled/Uncontrolled Components**: Most inputs are controlled.

---

## 1. **BookingCalendar/BookingCalendar.js**

**Potential Issues**:
- The line  
  ```jsx
  {slots[selectedDate]?.map((slot) => (
    ...
  )) || <div>No slots for this date.</div>}
  ```
  can result in **`No slots for this date.`** never being rendered, because if `slots[selectedDate]` is `undefined`, `.map()` will not execute but `undefined` is not a falsy value for `||` in JSX (since `undefined.map` errors!).  
**Fix**: Instead, do:
  ```jsx
  {Array.isArray(slots[selectedDate]) && slots[selectedDate].length > 0
    ? slots[selectedDate].map((slot) => (
        <BookingSlot key={slot.id} slot={slot} onClick={() => selectSlot(slot)} />
      ))
    : <div>No slots for this date.</div>
  }
  ```
**Also**:
- Assumes `selectedDate` is valid and set.
---

## 2. **BookingForm/BookingForm.js**

**Possible Bug**:  
- `CustomerInfo` and `RescheduleOptions` are being fed `value={data.customer}` and `value={data.reschedule}` but your `handleChange` only updates **top-level** fields (like `service`) and not deep objects (`customerName` within `customer`, etc).
- If these child components update fields like `customerName`, they will call `onChange`, which is your `handleChange`, but `e.target.name = customerName` and `bookingForm.data` doesn't have a top-level field called `customerName`.
- You would need a custom handler per subobject, or refactor your state structure for nesting.

**Potential Fix/Advice**:
- Pass `data.customerName`, `data.customerEmail`, etc. as top-level, and have `CustomerInfo` just work off those.
- Or, in handler, infer nesting based on the `name` field, e.g. `customer.customerName`.

**Example (simple workaround)**:
If you flatten the booking form state (single level for all inputs), and flatten the names:
```jsx
//... your form state
<CustomerInfo
  value={data}
  onChange={handleChange}
/>
```
and inside `CustomerInfo`, inputs are named e.g. `customerName`, `customerEmail`, etc.  
Then handleChange will work.

---

## 3. **BookingForm/CustomerInfo.js**

**Works as expected** if parent passes flat values as above.

---

## 4. **BookingForm/RescheduleOptions.js**

- `onChange` will get a change event from the checkbox and from the "rescheduleTime" input, both wired to same onChange in parent—be sure parent knows how to handle them.
- `checked={!!value.canReschedule}` is good practice.

---

## 5. **AIManager/AICommunication.js**

- `conversation.map()` assumes `conversation` is always an array.
- If `conversation` could be undefined/null, add:  
  `{(conversation ?? []).map(...)}`

---

## 6. **AIManager/BookingOptimizer.js**  
- No issues. Purely presentational.

---

## 7. **AIManager/ReminderScheduler.js**  
- `reminders` should always be an array to avoid runtime error.  
- Suggest:  
  `{(reminders || []).map(...)}`

---

## 8. **Payment/StripeProviderWrapper.js**

- Will work as placeholder. If real Stripe context needed, must uncomment actual code and install Stripe libraries.

---

## 9. **Common/Modal.js**

- When you click "✖", both the button and the backdrop will call `onClose`. Because you are **stopping propagation** on the modal content, this works as expected.
- No errors.

---

## 10. **Common/ErrorBoundary.js**

- Works fine. The error message display is graceful.

---

## **Other components:**  
No noteworthy code issues. All are functional.

---

# **Summary of Errors/Improvements**

1. **BookingCalendar.js**:  
   - The logic for displaying "No slots for this date." does NOT work as intended if there are 0 slots or `undefined`.  
   **Fix:** Use `Array.isArray(slots[selectedDate])` and length check.

2. **BookingForm.js, CustomerInfo.js, RescheduleOptions.js**:  
   - The way you structure the `data` and the onChange handlers will not correctly update **nested input fields** (e.g., `customer.customerName`) unless you either:
     - Ensure all fields are at the top level, **or**
     - Use a custom handler for each sub-object.  
   **Fix**: Consider flattening the form state, or using deep updating logic.

3. **AICommunication.js, ReminderScheduler.js**:  
   - If `conversation` or `reminders` might be `null/undefined`, should default to an empty array in `.map()` to avoid runtime errors.

---

## **No compile-time errors** found in other files. Props and imports are well handled.

---

## **Corrections (EXAMPLES):**

### BookingCalendar.js (fix "No slots" rendering):
```jsx
<div className="slots-grid">
  {Array.isArray(slots[selectedDate]) && slots[selectedDate].length > 0
    ? slots[selectedDate].map(slot =>
        <BookingSlot key={slot.id} slot={slot} onClick={() => selectSlot(slot)} />
      )
    : <div>No slots for this date.</div>
  }
</div>
```

### BookingForm.js (flatten state for customer fields):
- Change initial structure:
  ```jsx
  const [data, setData] = useState({
    customerName: '',
    customerEmail: '',
    customerPhone: '',
    service: '',
    // ...
  });
  ```
- Pass value/handlers directly in `CustomerInfo`.

### AICommunication.js (defensive `map`):
```jsx
{(conversation || []).map((msg, idx) => (/* ... */))}
```

---

**If you want refactored demo code for the fixed/more robust pieces, just say so!**

In [57]:
response = openai.chat.completions.create(
    messages=[
        {"role": "assistant", "content": "You're a helpful assistant"},
        {"role": "user", "content": f"""You're provide with with given source code \n {source}\n{results}\n and summary of improvements and errors that need to be fixed {tester_response}\nPlease fix the errors."""}
    ],
    model="gpt-4.1-mini",
)
refined_source = response.choices[0].message.content
display(Markdown(refined_source))


Certainly! Below are the fixes applied to the components where errors/issues were identified. For clarity, I include only the modified files with explanations.

---

## 1. Fix in **BookingCalendar/BookingCalendar.js**

**Problem:**
- The expression `{slots[selectedDate]?.map(...) || <div>No slots for this date.</div>}` will *not* render the fallback if `slots[selectedDate]` is `undefined` or empty array correctly, and may cause runtime errors if `slots[selectedDate]` is `undefined`.

**Fix:**
- Use explicit checks for array existence and length.

**Fixed BookingCalendar.js:**

```jsx
import React, { useContext } from 'react';
import { BookingContext } from '../../context/BookingContext';
import BookingSlot from './BookingSlot';
import DoubleBookingAlert from './DoubleBookingAlert';

const BookingCalendar = () => {
  const { slots, doubleBooking, selectSlot, selectedDate } = useContext(BookingContext);

  if (!slots) return <div>Loading calendar...</div>;

  const daySlots = Array.isArray(slots[selectedDate]) ? slots[selectedDate] : [];

  return (
    <div className="booking-calendar">
      <h2>Booking Calendar</h2>
      {doubleBooking && <DoubleBookingAlert />}
      <div className="slots-grid">
        {
          daySlots.length > 0 
            ? daySlots.map(slot => (
                <BookingSlot 
                  key={slot.id} 
                  slot={slot} 
                  onClick={() => selectSlot(slot)} 
                />
              ))
            : <div>No slots for this date.</div>
        }
      </div>
    </div>
  );
};

export default BookingCalendar;
```

---

## 2. Fix in **BookingForm/BookingForm.js** with related **CustomerInfo.js** and **RescheduleOptions.js**

### Problem:

- `BookingForm` state (`data`) has nested objects (e.g., `data.customer`), but `handleChange` works only on flat top-level keys.
- `onChange` handlers in `CustomerInfo` and `RescheduleOptions` assume change events with single key names, but the nested fields won’t update the state correctly.

### General Solution:

- Flatten the form data state at the `BookingForm` level so all fields live in one object.
- Pass down flat values and handlers accordingly.
- In `handleChange`, update the relevant field directly at top-level.
- In the child components, use the flat props.

---

### Updated **BookingForm.js**

```jsx
import React, { useState } from 'react';
import CustomerInfo from './CustomerInfo';
import RescheduleOptions from './RescheduleOptions';

const BookingForm = ({ initialData = {}, onSubmit }) => {
  // Flatten the initial data to handle nested fields correctly
  const [data, setData] = useState({
    customerName: initialData.customerName || '',
    customerEmail: initialData.customerEmail || '',
    customerPhone: initialData.customerPhone || '',
    service: initialData.service || '',
    canReschedule: initialData.canReschedule || false,
    rescheduleTime: initialData.rescheduleTime || '',
    // ... add other fields as needed
  });

  const handleChange = (e) => {
    const { name, type, value, checked } = e.target;
    setData(prevData => ({
      ...prevData,
      [name]: type === 'checkbox' ? checked : value,
    }));
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    // TODO: Add validation before submitting
    onSubmit(data);
  };

  // Pass relevant props to child components as flat values
  return (
    <form onSubmit={handleSubmit}>
      <CustomerInfo value={data} onChange={handleChange} />
      <input
        type="text"
        name="service"
        value={data.service}
        onChange={handleChange}
        placeholder="Service"
        required
      />
      <RescheduleOptions 
        value={{ 
          canReschedule: data.canReschedule, 
          rescheduleTime: data.rescheduleTime 
        }} 
        onChange={handleChange} 
      />
      <button type="submit">Book Now</button>
    </form>
  );
};

export default BookingForm;
```

---

### Updated **CustomerInfo.js**

```jsx
import React from 'react';

const CustomerInfo = ({ value = {}, onChange }) => (
  <div className="customer-info">
    <input
      type="text"
      name="customerName"
      value={value.customerName || ''}
      onChange={onChange}
      placeholder="Full Name"
      required
    />
    <input
      type="email"
      name="customerEmail"
      value={value.customerEmail || ''}
      onChange={onChange}
      placeholder="Email"
      required
    />
    <input
      type="tel"
      name="customerPhone"
      value={value.customerPhone || ''}
      onChange={onChange}
      placeholder="Phone"
      required
    />
  </div>
);

export default CustomerInfo;
```

---

### Updated **RescheduleOptions.js**

```jsx
import React from 'react';

const RescheduleOptions = ({ value = {}, onChange }) => (
  <div className="reschedule-options">
    <label>
      <input
        type="checkbox"
        name="canReschedule"
        checked={!!value.canReschedule}
        onChange={onChange}
      />
      Allow customer to request a reschedule
    </label>
    {value.canReschedule && (
      <input
        type="datetime-local"
        name="rescheduleTime"
        value={value.rescheduleTime || ''}
        onChange={onChange}
        placeholder="Pick new time"
      />
    )}
  </div>
);

export default RescheduleOptions;
```

---

## 3. Defensive improvements in **AICommunication.js** and **ReminderScheduler.js**

### Problem:
- If `conversation` or `reminders` props can be `null` or `undefined`, calling `.map` induces runtime errors.

### Fix: default to empty array before mapping.

---

### Updated **AICommunication.js**

```jsx
import React, { useState } from 'react';
import { useAI } from '../../hooks/useAI';

const AICommunication = ({ conversation }) => {
  const { sendMessage } = useAI();
  const [input, setInput] = useState('');

  const handleSend = async () => {
    if (!input.trim()) return; // optional: prevent sending empty
    await sendMessage(input);
    setInput('');
  };

  return (
    <div className="ai-communication">
      <h3>AI Chat</h3>
      <div className="conversation">
        {(conversation || []).map((msg, idx) => (
          <div key={idx} className={`msg ${msg.sender}`}>{msg.text}</div>
        ))}
      </div>
      <input
        value={input}
        onChange={e => setInput(e.target.value)}
        placeholder="Ask something..."
      />
      <button onClick={handleSend}>Send</button>
    </div>
  );
};

export default AICommunication;
```

---

### Updated **ReminderScheduler.js**

```jsx
import React from 'react';

const ReminderScheduler = ({ reminders }) => (
  <div className="reminder-scheduler">
    <h4>Upcoming Reminders</h4>
    <ul>
      {(reminders || []).map((r, i) => (
        <li key={i}>
          Reminder for {r.customerName}: {r.time}, via {r.channel}
        </li>
      ))}
    </ul>
  </div>
);

export default ReminderScheduler;
```

---

# Summary

- Fixed conditional slot rendering in `BookingCalendar` to avoid no rendering or runtime errors.
- Flattened form state and adapted handlers in `BookingForm` and nested components to properly handle controlled inputs that represent nested data.
- Added defensive defaults in `AICommunication` and `ReminderScheduler` components to avoid errors in `.map` iteration if props are not arrays.
- Optional improvement: Added check to prevent sending empty messages in AI chat.

If you want, I can also provide refactored versions of other components or full files with these fixes integrated. Just let me know!